In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\DELL\\projects\\end-to-end-ML-with-MLflow\\research'

In [3]:
os.chdir("../") # go back one step 

In [4]:
%pwd

'c:\\Users\\DELL\\projects\\end-to-end-ML-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path 

# in python calss actually if you are defining class variables you need to give self .
# if you want to define the variable you can use something called Data class 
#the same thing in yaml (variable will return)
#it's return data class 
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path 




In [6]:
from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories 

class ConfigurationManager: 
    def __init__ (self, config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH): 

        self.config = read_yaml (config_filepath)
        self.params = read_yaml (params_filepath)
        self.schema = read_yaml (schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        print(config)
        print(config.root_dir)
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL = config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    

In [7]:
import os 
import urllib.request as request 
import zipfile 
from mlproject import logger 
from mlproject.utils.common import get_size 


In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig): 
        self.config = config 

    def download_file(self): 
        if not os.path.exists(self.config.local_data_file): 
            filename, headers = request.urlretrieve( 
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self): 
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref: 
            zip_ref.extractall(unzip_path)




In [9]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e : 
    raise e 

[2024-12-29 21:58:41,316 - common - INFO - yaml file: config\config.yaml loadded successfully]
[2024-12-29 21:58:41,318 - common - INFO - yaml file: params.yaml loadded successfully]
[2024-12-29 21:58:41,320 - common - INFO - yaml file: schema.yaml loadded successfully]
[2024-12-29 21:58:41,321 - common - INFO - created directory at: artifacts]
{'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/Ahmad-Amireh/end-to-end-ML-with-MLflow/raw/refs/heads/main/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}
artifacts/data_ingestion
[2024-12-29 21:58:41,322 - common - INFO - created directory at: artifacts/data_ingestion]
[2024-12-29 21:58:42,362 - 3548988504 - INFO - artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: applicati